# Use MDA(mean decrease accuracy) to select features
* this notebook will use MDA(mean decrease accuracy) on random forest to select the features that contributes the most to classfication scores.
* The goal is to be able to do feature selection and find the most important features.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from urllib.request import urlopen

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)

In [2]:
%run Batch_ArtifactFilter_Epoch.ipynb

%run .../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../../preprocessing/Artifact_Removal/Extract_Describer_Events.ipynb
%run ../../preprocessing/StimCodes.ipynb
%run ../../Classification/ConcatEpochTrails.ipynb

ERROR:root:File `'.../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb.py'` not found.
ERROR:root:File `'../preprocessing/StimCodes.ipynb.py'` not found.
ERROR:root:File `'../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb.py'` not found.
ERROR:root:File `'../preprocessing/StimCodes.ipynb.py'` not found.


In [4]:
epoch=mne.read_epochs('D:\\USB Drive\\NewEEG-200s\\New\\20131216_1310_epo.fif')

Reading D:\USB Drive\NewEEG-200s\New\20131216_1310_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1759 matching events found
Applying baseline correction (mode: mean)
1759 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [5]:

## Methods for convert our events into binary, 4: Language, 5:Non Language
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=5
        else:
            for i in range(len(events)):
                    events[i]=4
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [6]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

[5 5 5 ... 5 5 5]


In [6]:
## Drop Bad Channels
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG','STI 014']);

In [ ]:
## Vecorized 3d to 2d
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

In [24]:
## Logisticalregression classfier
clf = make_pipeline(Vectorizer(),
                   StandardScaler(),
                         LogisticRegression())
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
score = clf.score(X_test, y_test)
print(score)

C:\Users\kingsly\Anaconda3\envs\mne\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5549242424242424


In [25]:
y_pred

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [26]:
def get_epoch_trials(epoch_object):
    #note: third stim-code has a value that is 1 more than the first.
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    for event in events:
        if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
            current_trial.append(event)
            found_first_stim_code = True
            first_stim_code = event
        elif not found_second_stim_code and not found_third_stim_code:
            current_trial.append(event)
            found_second_stim_code = True
        elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
            current_trial.append(event)
            found_third_stim_code = True
            trials.append(current_trial)
            current_trial = []
            found_first_stim_code = False
            found_second_stim_code = False
            found_third_stim_code = False
        elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
            # there is no third event, trial is unfinished.
            current_trial = []
            current_trial.append(event)#assuming it's first stim-code...for now
            found_first_stim_code = True
    return trials

In [28]:
def stim_code_combination_from_trial(stim_code_tuple, stim_combinations):
    """
    Description:
        returns the correct stim-code combination event id 
        example - a trial has events (20, 15, 19), this method will match those values to a 
        stim combination that is defined in the dictionary stim_combinations
    """
    for stim_code in stim_combinations.keys():
        if stim_code_tuple == stim_code:
            return stim_combinations[stim_code]
    return "NaN"

In [29]:
def convert_epoch_events_to_stim_combinations(epoch_object):
    trials = get_epoch_trials(epoch_object)
    new_events_master = []
    
    for trial in trials:
#         print(trial)
        trial = np.array(trial)
        stim_codes = trial[:,-1]
        try:
#             print(stim_codes)
            stim_code_tuple = (stim_codes[0],stim_codes[1], stim_codes[2])
#             print(stim_code_tuple)
            stim_combination = stim_code_combination_from_trial(stim_code_tuple, stim_combinations)

            new_event_id = modality_lexicality_event_ids[stim_combination]
            trial[:,-1] = new_event_id
            new_events_master.append(trial)
        except:
            print("Error with stim-code: {0}".format(stim_codes))
        
    return np.concatenate([x for x in new_events_master])

In [38]:
new_events = convert_epoch_events_to_stim_combinations(epoch)    

Error with stim-code: [11 12]


In [37]:
stim_combinations = {
    (5,6,4): "AALL",
    (17,18,16) :"AALL",
    
    (11,6,10):"AALN",
    (23,18,22):"AALN",
    
    (5,12,4) :"AANL",
    (17,24,16):"AANL",
    
    (11,12,10) : "AANN",
    (23,24,22): "AANN",
    
    (2,6,1):"AVLL",
    (14,18,13):"AVLL",
    
    (8,12,7):"AVNN",
    (20,24,19):"AVNN",
    
    (5,3,4) :"VALL",
    (17,15,16):"VALL",
    
    (11,9,10) :"VANN",
    (23,21,22):"VANN",
    
    (2,3,1) :"VVLL",
    (14,15,13):"VVLL",
    
    (8,3,7):"VVLN",
    (20,15,19):"VVLN",
    
    (2,9,1) :"VVNL",
    (14,21,13):"VVNL",
    
    (8,9,7) :"VVNN",
    (20,21,19):"VVNN"
}

In [39]:
new_events

array([[   464,      0,    512],
       [   713,      0,    512],
       [   972,      0,    512],
       ...,
       [614661,      0,    912],
       [614920,      0,    912],
       [614922,      0,    912]])

In [40]:
copy_epoch = epoch.copy()

In [48]:
X = copy_epoch.pick_types(meg=False, eeg=True)

In [42]:
copy_epoch.events = new_events

In [49]:
copy_epoch.events[:,-1].shape

(1116,)

In [68]:
X.shape

(1759, 31868)

In [ ]:
X.get_data().shape

In [ ]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

In [7]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG','STI 014']);

In [8]:
X = epoch.get_data()
y = epoch.events[:,-1]
print(X.shape,epoch.get_data().shape)

(1759, 124, 257) (1759, 124, 257)


In [9]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel

In [ ]:
## Use Mean Decrease Accuracy to select important features and fit the data.
sel = SelectFromModel(
    PermutationImportance(RandomForestRegressor(n_estimators =10, random_state = 42),cv=5),
    threshold=0.05,
).fit(X, y)


In [ ]:
X_trans = sel.transform(X)

In [ ]:
clf = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators =10, random_state = 42))
time_decod = SlidingEstimator(clf, n_jobs=1, scoring='roc_auc')
time_decod.fit(X, y)

In [ ]:
coef = time_decod.feature_importances
evoked = mne.EvokedArray(coef, epochs.info, tmin=epochs.times[0])
joint_kwargs = dict(ts_args=dict(time_unit='s'),
                    topomap_args=dict(time_unit='s'))
evoked.plot_joint(times=np.arange(0., .500, .100), title='patterns',
                  **joint_kwargs)